# Reddit Scraping
In order to scrape posts from Reddit, we used the [`praw` python library](https://github.com/praw-dev/praw). 
In order to use that, we need:
- a Reddit account.
- a Reddit app ([here](https://www.reddit.com/prefs/apps) you can create one). Here you'll find the client id under the app name once you create it, as well as the client secret and the user agent, which is your app name.

Once we have those, we can access reddit from the terminal using these steps(it is also possible to access as a user by adding username and account if you want to use it in write mode):

In [1]:
#pip install praw
import praw
reddit = praw.Reddit(client_id='', client_secret='g', user_agent='')

We can analyse posts, post requirements etc. for a single subreddit or for all of them(by using the subreddit 'all'); we can also select top posts or do searches. 
# sistemare data utc

In [2]:
for i in reddit.subreddit('all').top(limit=5): #this will print the top posts all time
    print(i.title + ' Author: ' + i.author.name +  ' Link: https://www.reddit.com' + i.permalink + ' Subreddit: r/' + i.subreddit.display_name + ' Upvote Ratio: ' + str(i.upvote_ratio) + ' Date: '+str(i.created_utc))


I’ve found a few funny memories during lockdown. This is from my 1st tour in 89, backstage in Vegas. Author: ReallyRickAstley Link: https://www.reddit.com/r/pics/comments/haucpf/ive_found_a_few_funny_memories_during_lockdown/ Subreddit: r/pics Upvote Ratio: 0.99 Date: 1592410647.0
Times Square right now Author: SomeGuyInDeutschland Link: https://www.reddit.com/r/wallstreetbets/comments/l8rf4k/times_square_right_now/ Subreddit: r/wallstreetbets Upvote Ratio: 0.99 Date: 1612029638.0
Joe Biden elected president of the United States Author: throwawaynumber53 Link: https://www.reddit.com/r/news/comments/jptqj9/joe_biden_elected_president_of_the_united_states/ Subreddit: r/news Upvote Ratio: 0.88 Date: 1604766517.0
The Senate. Upvote this so that people see it when they Google "The Senate". Author: serventofgaben Link: https://www.reddit.com/r/movies/comments/62sjuh/the_senate_upvote_this_so_that_people_see_it_when/ Subreddit: r/movies Upvote Ratio: 0.96 Date: 1491051474.0
My cab driver toni

# Inserire spiegazione differenza tra le diverse scale hot top controversial
Now, for our analysis we will use data from the subreddit r/conspiracy and we will try to study the ramifications of the posts.

In [3]:
'''import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

#put the subreddit already passed here
id_to_avoid = [reddit.subreddit('conspiracy').id]

for i in reddit.subreddit("conspiracy").hot(limit=5): #recupera i 5000 post hot di conspiracy
    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + l'url che viene postato


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]): #trova quando il post è stato citato
        if repost.subreddit_id not in id_to_avoid:
            
            subreddit_url = str(repost.subreddit)
            subreddit_url = "https://www.reddit.com/r/" + subreddit_url
            if subreddit_url in conspiracy_dict.keys():
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1][0] +=1
            else:
                conspiracy_dict[subreddit_url]=[[],[1]]
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_hot_url.csv',index=False)'''

'import pandas as pd\npost_list=list()\nsubreddit_list = list()\nconspiracy_dict=dict()\n\n#put the subreddit already passed here\nid_to_avoid = [reddit.subreddit(\'conspiracy\').id]\n\nfor i in reddit.subreddit("conspiracy").hot(limit=5): #recupera i 5000 post hot di conspiracy\n    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + l\'url che viene postato\n\n\nfor post in post_list:\n    for repost in reddit.subreddit(\'all\').search(\'url:\'+post[2]): #trova quando il post è stato citato\n        if repost.subreddit_id not in id_to_avoid:\n            \n            subreddit_url = str(repost.subreddit)\n            subreddit_url = "https://www.reddit.com/r/" + subreddit_url\n            if subreddit_url in conspiracy_dict.keys():\n                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)\n                conspiracy_dict[subreddit_url][1][0] +=1\n            else:\n                conspiracy_dict[subreddit_url]=

An interesting thing we noted is that a good part of the propagation of news comes from bots, especially from the top posts.

In [4]:
'''
Da fare: script che accumula tutti gli id nella cartella
Let's suppose that the results are in "results/conspiracy_hot.csv". 
If you have multiple csv in the directory "my_directory", you may uncomment this and use the for loop in here:
import os
dir = "my_directory"
to_scan = [] #iterate over this
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and f[-4:] == ".csv":
        to_scan.append(f) 
id_to_analyse = []

for file in to_scan:
    df1 = pd.read_csv('results/conspiracy_hot.csv')
    for column in df1.columns:
        value = df1[column][1][1:-1]
        if int(value) >= 5:
            ind = column.index('/r/')
            id = column[ind+3:]
            id_to_analyse.append(id)


df1 = pd.read_csv('results/conspiracy_hot.csv')
id_to_analyse = []
for column in df1.columns:
    value = df1[column][1][1:-1]
    if int(value) >= 5:
        ind = column.index('/r/')
        id = column[ind+3:]
        id_to_analyse.append(id)

for subreddit in id_to_analyse:
    post_list=list()
    subreddit_list = list()
    conspiracy_dict=dict() 
    for i in reddit.subreddit(subreddit).top(limit=500):
        post_list.append((i.title, i.score, "https://www.reddit.com"+i.permalink))


    for post in post_list:
        for repost in reddit.subreddit('all').search('url:'+post[2]):
            if repost.subreddit_id != subreddit: #cosa facciamo?
                subreddit_url = str(repost.subreddit)
                subreddit_url = "https://www.reddit.com/r/" + subreddit_url
                if subreddit_url in conspiracy_dict.keys():
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                    conspiracy_dict[subreddit_url][1][0] +=1
                else:
                    conspiracy_dict[subreddit_url]=[[],[1]]
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
    df = pd.DataFrame(conspiracy_dict)
    df.to_csv(r'results/'+subreddit+'.csv')
    '''


'\nDa fare: script che accumula tutti gli id nella cartella\nLet\'s suppose that the results are in "results/conspiracy_hot.csv". \nIf you have multiple csv in the directory "my_directory", you may uncomment this and use the for loop in here:\nimport os\ndir = "my_directory"\nto_scan = [] #iterate over this\nfor filename in os.listdir(directory):\n    f = os.path.join(directory, filename)\n    if os.path.isfile(f) and f[-4:] == ".csv":\n        to_scan.append(f) \nid_to_analyse = []\n\nfor file in to_scan:\n    df1 = pd.read_csv(\'results/conspiracy_hot.csv\')\n    for column in df1.columns:\n        value = df1[column][1][1:-1]\n        if int(value) >= 5:\n            ind = column.index(\'/r/\')\n            id = column[ind+3:]\n            id_to_analyse.append(id)\n\n\ndf1 = pd.read_csv(\'results/conspiracy_hot.csv\')\nid_to_analyse = []\nfor column in df1.columns:\n    value = df1[column][1][1:-1]\n    if int(value) >= 5:\n        ind = column.index(\'/r/\')\n        id = column[in

# Sentiment analysis
To do sentiment analysis, we will use the syuzhet package in R.
First we need to extract the comments inside a Reddit post. We will ignore the comments of bots, since they do not provide information about how the user interacts with it, even though it can interfere with it.
https://realpython.com/python-nltk-sentiment-analysis/
Caveat: \s indica il sarcasmo

In [2]:
import random #prendi un sample
def iterSample(iterable, samplesize):
    results = []
    for i, v in enumerate(iterable):
        r = random.randint(0, i)
        if r < samplesize:
            if i < samplesize:
                results.insert(r, v) # add first samplesize items in random order
            else:
                results[r] = v # at a decreasing rate, replace random items

    if len(results) < samplesize:
        raise ValueError("Sample larger than population.")

    return results

In [1]:
#pip install praw
import praw
reddit = praw.Reddit(client_id='', client_secret='', user_agent='')
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

# BY using the following snippet we get all the used files inside the folders

In [2]:
import pandas as pd
dir = "results/1st_level"
top_comments_list = dict()
to_scan = [file for file in get_all_in_dir("results/1st_level")]
to_avoid= [file.split('\\')[1] for file in get_all_in_dir("results/1st_level")]
to_avoid.extend(file.split('\\')[1] for file in get_all_in_dir("results/1st_level_2"))
to_avoid.extend(file.split('\\')[1] for file in get_all_in_dir("results/2nd_level"))

for subr in to_scan:
        id_to_analyse = []
        print(f'now opening {subr}')
        try:
                df1 = pd.read_csv(subr)
        except:
                pass
        try:
                df1 = pd.read_csv(subr, encoding='utf8')
        except:
                print(f'unable to open {subr}')
                continue
        for column in df1.columns:
                try: 
                        value = df1[column][1][1:-1]
                except:
                        continue
                if int(value) >= 5:
                        ind = column.index('/r/')
                        id = column[ind+3:]
                        if not id + '.csv' in to_avoid:
                                try:
                                        post_list=list()
                                        subreddit_list = list()
                                        conspiracy_dict=dict() 
                                        for i in reddit.subreddit(id).top(limit=500):
                                                post_list.append((i.title, i.score, i.url))

                                        for post in post_list:
                                                for repost in reddit.subreddit('all').search('url:'+post[2]):
                                                        if repost.subreddit_id != "t5_"+reddit.subreddit(id).id: #cosa facciamo?
                                                                subreddit_url = str(repost.subreddit)
                                                                subreddit_url = "https://www.reddit.com/r/" + subreddit_url
                                                        else:
                                                                continue
                                                        if subreddit_url in conspiracy_dict.keys():
                                                                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                                                                conspiracy_dict[subreddit_url][1][0] +=1
                                                        else:
                                                                conspiracy_dict[subreddit_url]=[[],[1]]
                                                                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                                        
                                        df = pd.DataFrame(conspiracy_dict)
                                        df.to_csv(r'results/2nd_level/'+id+'.csv')
                                        with open("results/2nd_level/done.txt",'a', encoding = "utf-8") as text_note:
                                                text_note.write(id + "\n")
                                                text_note.close()
                                        
                                except Exception as E:
                                        print(E)
                                        with open("results/2nd_level/error.txt",'a', encoding = "utf-8") as text_note:
                                                text_note.write(id + "\n")
                                                text_note.close()
                                

now opening results/1st_level\1984isreality.csv


In [1]:
import pandas as pd
import traceback
import re
top_comments_list = []
import os
dir = "results/"
to_scan = [file for file in get_all_in_dir('results/1st_level')] #iterate over this

to_scan.extend(file for file in get_all_in_dir('results/2nd_level'))

for subreddit in to_scan:
        try:
                df = pd.read_csv(subreddit, sep=';', encoding='unicode_escape')
        except:
                pass
        try:
                df = pd.read_csv(subreddit, sep=',', encoding='utf8')
        except:
                pass
        try:
                df = pd.read_csv(subreddit, sep=';', encoding='utf8')
        except:
                pass
        try:
                df = pd.read_csv(subreddit, sep=',', encoding='unicode_escape')
        except:
                traceback.print_exc()
                print(f'{subreddit} should be opened with different settings.')
                continue
        for column in df.columns: # qui inserire un loop sui permalink
                list_post = []
                try:
                        list_post= df[column][1].split(',')
                except:
                        print(f'{subreddit} has problems in formatting.')
                        pass
                for post_url in list_post: #rifai con regex
                        tmp = post_url
                        tmp = re.sub("'[]", '', tmp)
                        
                        try:
                                post = reddit.submission(url=tmp)
                        except: # some posts may be removed
                                print(f'{tmp} was not accessible')
                                continue
                        try:
                                post.comments.replace_more(limit=2)
                        except:
                                print(f'{tmp} was not accessible')
                                continue
                        result = []
                        if len(post.comments)> 500:
                                for comment in iterSample(post.comments, 500):
                                        result.append(comment)
                        else:
                                for comment in post.comments:
                                        result.append(comment)
                        result={'permalink':tmp,'comments': result, 'target': df[column][0], 'posted_from':subreddit}
                        top_comments_list.append(result)


NameError: name 'get_all_in_dir' is not defined

# Sentiment Analysis


In [17]:
import nltk
nltk.download(['stopwords', "vader_lexicon"])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [18]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from statistics import mean
sent = SentimentIntensityAnalyzer()
sentiment_values = list()
for post in top_comments_list:
    post_sentiment = [] # median sentiment

    for comment in post['comments']:
        body = comment.body
        if not "I'm a bot" in body and not 'I am a bot' in body:
            for reply in comment.replies:
                if not "I'm a bot" in reply.body and not 'I am a bot' in reply.body:
                    body += reply.body
                for second_reply in reply.replies:
                    if not "I'm a bot" in second_reply.body and not 'I am a bot' in second_reply.body:
                        body += second_reply.body
            val = sent.polarity_scores(body)['compound']
            post_sentiment.append(val)
    try:
        post_sentiment = mean(post_sentiment)
    except:
        post_sentiment = 0
    sentiment_values.append({'sentiment': post_sentiment,'permalink': post['permalink'],'posted_in': post['target'], 'posted_from':post['posted_from']})

In [19]:
sentiment_df = pd.DataFrame(sentiment_values)
sentiment_df.to_csv('results/sentiment/sentiment.csv')

In [26]:
for subreddit in to_scan:
        try:
                df = pd.read_csv(subreddit, sep=';', encoding='unicode_escape')
        except:
                pass
        try:
                df = pd.read_csv(subreddit, sep=',', encoding='unicode_escape')
        except:
                traceback.print_exc()
                print(f'{subreddit} should be opened with different settings.')
                continue